<a href="https://colab.research.google.com/github/silvamt/dio_assistente_voz/blob/main/assistente_voz.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 0. instalacoes e imports

In [7]:
from IPython.display import Audio, Javascript, display
import numpy as np
import base64 # Adicionando a importação do módulo base64
from google.colab import output # Certifique-se de que 'output' está importado aqui

!pip install pydub
!apt-get install -y ffmpeg

import speech_recognition as sr
import base64
import io
from pydub import AudioSegment
import tempfile
import os

from google.colab import ai

!pip install gtts
from gtts import gTTS
from IPython.display import Audio
import io


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 1 not upgraded.


# 1. captacao do audio do microfone

In [8]:
# Global variable to store the base64 audio data
recorded_audio_base64 = None

# Python function to receive the base64 audio data
def process(b64_audio_data):
    global recorded_audio_base64
    recorded_audio_base64 = b64_audio_data
    print("Áudio recebido e armazenado em 'recorded_audio_base64'.")
    # Display the audio player
    display(Audio(data=base64.b64decode(b64_audio_data)))

# Captar áudio com interface simples do Colab
def gravar_audio_colab(duracao=5):
    """Interface simples para gravar áudio no Colab"""

    js_code = '''
    (async () => { // Usamos uma IIFE (Immediately Invoked Function Expression)
        console.log('Iniciando gravação de áudio...');
        const stream = await navigator.mediaDevices.getUserMedia({audio: true});
        const mediaRecorder = new MediaRecorder(stream, { mimeType: 'audio/webm' }); // Especifica o tipo como webm
        const audioChunks = [];

        mediaRecorder.ondataavailable = (event) => audioChunks.push(event.data);

        // Retorna uma Promise que será resolvida com os dados de áudio base64
        const audioPromise = new Promise((resolve) => {
            mediaRecorder.onstop = () => {
                console.log('Gravação parada. Criando blob de áudio...');
                const audioBlob = new Blob(audioChunks, {type: 'audio/webm'}); // Alterado para webm
                const reader = new FileReader();
                reader.onloadend = () => {
                    const base64 = reader.result.split(',')[1];
                    console.log('Dados Base64 gerados. Tamanho:', base64.length);
                    resolve(base64); // Resolve a promise com os dados base64
                };
                reader.readAsDataURL(audioBlob);
            };
        });

        mediaRecorder.start();
        setTimeout(() => mediaRecorder.stop(), ''' + str(duracao * 1000) + ''');

        return audioPromise; // Retorna a promise da IIFE
    })()
    '''
    # Executa o JavaScript e obtém a string base64 de volta para o Python
    b64_audio_data = output.eval_js(js_code)
    # Agora chama a função Python 'process' diretamente com os dados recebidos
    process(b64_audio_data)

In [25]:
audio = gravar_audio_colab()

Áudio recebido e armazenado em 'recorded_audio_base64'.


# 2. audio em texto

In [10]:

def audio_para_texto(audio_data_base64):
    recognizer = sr.Recognizer()
    temp_audio_file = None
    try:
        # Decodificar o áudio base64
        audio_bytes = base64.b64decode(audio_data_base64)
        audio_io = io.BytesIO(audio_bytes)

        # Usar pydub para carregar o áudio (assumindo webm agora)
        audio_segment = AudioSegment.from_file(audio_io, format="webm") # Alterado para webm

        # Salvar em um arquivo WAV temporário, que é o formato preferido pelo SpeechRecognition
        with tempfile.NamedTemporaryFile(suffix=".wav", delete=False) as tf:
            temp_audio_file = tf.name
            audio_segment.export(temp_audio_file, format="wav")

        # Carregar o arquivo WAV temporário com SpeechRecognition
        with sr.AudioFile(temp_audio_file) as source:
            audio = recognizer.record(source)

        # Reconhecer o áudio usando o Google Web Speech API (ou outro)
        text = recognizer.recognize_google(audio, language='pt-BR')
        return text
    except sr.UnknownValueError:
        return "Não foi possível entender o áudio"
    except sr.RequestError as e:
        return f"Erro na requisição ao serviço de reconhecimento de fala; {e}"
    except Exception as e:
        return f"Ocorreu um erro inesperado: {e}"
    finally:
        # Limpar o arquivo temporário
        if temp_audio_file and os.path.exists(temp_audio_file):
            os.remove(temp_audio_file)

In [26]:
if recorded_audio_base64:
    texto_transcrito = audio_para_texto(recorded_audio_base64)
    print("Texto Transcrito:", texto_transcrito)
else:
    print("Nenhum áudio foi gravado ainda. Por favor, grave o áudio primeiro.")

Texto Transcrito: Por que o natal é comemorado no dia 25 de dezembro


# 3. texto ao gemini

In [27]:
response = ai.generate_text(f'{texto_transcrito}? Linguagem simples. Não use negrito. Não use emoji.')
print(response)

A Bíblia não conta o dia exato em que Jesus nasceu.

Então, por que 25 de dezembro?

Muito antes de existir o Natal como conhecemos, as pessoas no Império Romano já faziam grandes festas nessa época do ano. Isso acontecia porque era o fim do inverno no hemisfério norte, um tempo de renovação, e eles celebravam o "nascimento do Sol invencível" ou outras festas como a Saturnália.

Quando o Cristianismo começou a crescer, a Igreja Cristã percebeu que seria mais fácil para as pessoas aceitarem a nova religião se pudessem continuar celebrando em uma data já familiar.

Então, decidiram que o nascimento de Jesus seria comemorado no dia 25 de dezembro. Assim, o dia que já era de festa para alguns se tornou a celebração do nascimento de Cristo para os cristãos.

É mais uma escolha estratégica e cultural do que a data real do nascimento.


# 4. resposta gemini em voz

In [28]:
# Converter o texto em fala
tts = gTTS(text=response, lang='pt')

# Salvar o áudio em um objeto BytesIO
fp = io.BytesIO()
tts.write_to_fp(fp)
fp.seek(0)

# Reproduzir o áudio
Audio(fp.read(), autoplay=True)